<a href="https://colab.research.google.com/github/eduardova28/2025_UTEC_Programacion101/blob/main/TrabajoFinal_IA_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>